In [109]:
import os
import numpy as np
import pandas as pd

from keras import backend as K
from keras.activations import exponential as exp
from keras.models import Sequential
from keras.layers import Input,Dense
from keras.losses import CategoricalCrossentropy
from sklearn.preprocessing import OneHotEncoder

In [110]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Data Preprocessing

In [111]:
training_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Thesis/datasets/mnist/mnist_train.csv', header= None)
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Thesis/datasets/mnist/mnist_test.csv',header= None)

In [112]:
y_train_values = (training_data[784]).to_numpy()      # taking the last column where y labels are stored
y_train_values = np.reshape(y_train_values,(60000,1)) # changing the dimension of y labels numpy array

y_test_values = (test_data[784]).to_numpy()           # same for the testing data y labels
y_test_values = np.reshape(y_test_values,(10000,1))

print(y_train_values[0:6])
print(y_train_values.shape)
print(y_test_values[0:6])
print(y_test_values.shape)

[[5]
 [0]
 [4]
 [1]
 [9]
 [2]]
(60000, 1)
[[7]
 [2]
 [1]
 [0]
 [4]
 [1]]
(10000, 1)


In [113]:
onehot_encoder = OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(y_train_values) # changing training y labels to one hot encoding 
y_test = onehot_encoder.fit_transform(y_test_values)   # changing testing y labesl to one hot encoding 

print(y_train[0:6])
print(y_train.shape)
print(y_test[0:6])
print(y_test.shape)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
(60000, 10)
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
(10000, 10)


In [114]:
x_train = training_data.drop(784,axis=1)    # dropping the last column (y labels) from training x values
x_test = test_data.drop(784,axis=1)     

print(x_train.loc[0,260])
print(x_test.loc[0,260])

80
72


In [ ]:
# x_train.insert(0,0,np.ones((60000,1)),allow_duplicates = True)
# x_train = x_train.T.reset_index(drop=True).T

# x_test.insert(0,0,np.ones((10000,1)),allow_duplicates = True)
# x_test = x_test.T.reset_index(drop=True).T

# print(x_train.loc[0,261])
# print(x_test.loc[0,261])

In [115]:
x_train = x_train.values                  # changing pandas dataframe to numpy array
x_test = x_test.values
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


# Using Keras to build NN Model


In [134]:
# define the NN model with 256 hidden unit

def NN_model():
  model = Sequential()
  model.add(Dense(256,input_dim = 784,activation='sigmoid'))
  model.add(Dense(10,activation='softmax'))

  model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  return model

NN = NN_model()

In [135]:
ini_weights = NN.get_weights()
ini_weights_1_b = ini_weights[1]
ini_weights_1_n = ini_weights[0]
ini_weights_2_b = ini_weights[3]
ini_weights_2_n = ini_weights[2]

In [136]:
ini_weights_1_b.shape,ini_weights_1_n.shape,ini_weights_2_b.shape, ini_weights_2_n.shape

((256,), (784, 256), (10,), (256, 10))

In [137]:
ini_weights_1 = np.insert(ini_weights_1_n,0,ini_weights_1_b,axis=0)
ini_weights_2 = np.insert(ini_weights_2_n,0,ini_weights_2_b,axis=0)
print(ini_weights_1.shape)
print(ini_weights_2.shape)

(785, 256)
(257, 10)


In [138]:
x_sanity = x_train[0:6]
# x_sanity = np.reshape(x_sanity,(1,784))
y_sanity = y_train[0:6]
# y_sanity = np.reshape(y_sanity,(1,10))

x_sanity.shape, y_sanity.shape

((6, 784), (6, 10))

In [139]:
NN_model = NN.fit(x_sanity, y_sanity, batch_size = 1,epochs=1)

6/6 [==============================] - 0s 4ms/step - loss: 2.5822 - categorical_accuracy: 0.0000e+00


In [140]:
print(NN_model.history.keys())

dict_keys(['loss', 'categorical_accuracy'])


In [ ]:
# accuracy = NN_model.history['categorical_accuracy']
# loss = NN_model.history['loss']
# epochs = range(len(accuracy))

# fig = plt.figure(figsize=(10,8)) 
# plt.plot(epochs, accuracy, 'b', label = 'Training accuracy')
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.title('Training Accuracy')
# # txt = "Fig 1.b : "               # captioning the figure
# # fig.text(.5, .05, txt, ha='center')
# plt.legend()
# plt.show()


# fig = plt.figure(figsize=(10,8))
# plt.plot(epochs, loss, 'b', label = 'Training loss')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.title("Training Loss")
# plt.legend()

# plt.show()


In [141]:
weights = NN.get_weights()
weights_1_b = weights[1]
weights_1_n = weights[0]
weights_2_b = weights[3]
weights_2_n = weights[2]

weights_1 = np.insert(weights_1_n,0,weights_1_b,axis=0)
weights_2 = np.insert(weights_2_n,0,weights_2_b,axis=0)
print(weights_1.shape)
print(weights_2.shape)

(785, 256)
(257, 10)


In [146]:
print(ini_weights_1[:,4][0:5])
print(weights_1[:,4][0:5])

print(ini_weights_2[:,4][0:5])
print(weights_2[:,4][0:5])

[ 0.          0.0483203   0.04777004 -0.0335369  -0.02702244]
[-0.00316222  0.0483203   0.04777004 -0.0335369  -0.02702244]
[ 0.         -0.13938355 -0.02286084 -0.13783363  0.03205279]
[-0.00550895 -0.13945529 -0.02829796 -0.13790636  0.02425556]


In [147]:
NN.evaluate(x_test,y_test)

313/313 [==============================] - 1s 1ms/step - loss: 2.6864 - categorical_accuracy: 0.1306


[2.6864044666290283, 0.1306000053882599]

# Building NN without any package to validate the update functions


In [148]:
def sigmoid(x):
  return (1.0/(1.0 + np.exp(-x)))

def sigmoidD(x):
  return(x * (1-x))

def softmax(x = [ ]):
  s = sum(np.exp(x))
  x = np.exp(x)
  arr = x/s
  return arr

def forwardPass(x, weight, s):
    a = np.dot(x,weight)
    if (s == "sigmoid"):
      return sigmoid(a)
    elif (s == "softmax"):
      return softmax(a)


# trainError = 0.0
# loss = 0.0 

# # cce = CategoricalCrossentropy()
# # loss = cce(y_sanity[i],y_hat)
# # print(loss.numpy())
# for i in range(n):
#   z = fpass(x_built_sanity[i],ini_weights_1,"sigmoid")
#   z = np.insert(z,0,1,axis =0)
#   y_hat = fpass(z,ini_weights_2,"softmax")
#   error = -np.dot(y_sanity[i],np.log(y_hat))
#   print(error)
#   trainError += error

# print(trainError)

def full_forwardPass(w_1, w_2, x, y):
    n = x.shape[0] 

    y_pred = [] 

    # error = 0.0

    for i in range(n):
      z = forwardPass(x[i],w_1,"sigmoid")
      z = np.insert(z,0,1,axis =0)

      y_hat = forwardPass(z,w_2,"softmax")
      y_pred.append(y_hat)

      # example_error = -np.dot(y[i],np.log(y_hat))
      # error += example_error

    return np.array(y_pred)   


def calculate_Accuracy(y_hat,y_test_values, y_test):
  
  y_hat_label = np.argmax(y_hat,axis=1)
  
  count = 0
  loss = 0.0
  
  for i in range(y_hat.shape[0]):
    if (y_hat_label[i] == y_test_values[i]): 
      count+= 1
  
  accuracy = (count/y_hat.shape[0])
  loss = -np.sum(np.multiply(y_test, np.log(y_hat)))/y_hat.shape[0]

  return accuracy, loss


In [156]:
def multiTrain(w_1,w_2,x,y):
  iteration = 0                            # number of iteration
  eta = NN.optimizer.lr                    # taking learning rate as 0.001
  training_error = np.array([])            # declaring a numpy array to store the errors
  
  while (iteration < 1):
    print('Iteration number: % d' %(iteration))
    iteration +=1
    # error = compute_multiError(w_1, w_2, x, y)
    error = 0.00

    N = x.shape[0]
    
    for e in range(N):
      print('Exmaple Number: % d' %(e))

      z = forwardPass(x[e,:], w_1 ,"sigmoid")
      z = np.insert(z, 0, 1, axis = 0)
      y_hat = forwardPass(z, w_2,"softmax")
      
      error += -np.dot(y[e,:],np.log(y_hat))

      average_error = error/(e+1)

      
      print('Error of example %d: %f' %(e, average_error))
      
      delta_k = np.array([])

      for k in range(w_2.shape[1]):
        temp_delta_k = (y_hat[k]- y[e,k])
        delta_k = np.append(delta_k, temp_delta_k)
        for j in range(w_2.shape[0]):
          w_2[j,k] = w_2[j,k] + eta * temp_delta_k * z[j]

      # print('delta_k:')
      # print(delta_k)
      # print(delta_k.shape)

      print(w_2[:,4][0:5])

      delta_j = np.array([])

      for j in range(w_1.shape[1]):
        temp_delta_j = np.sum(np.dot(delta_k,w_2[j,:])) * sigmoidD(z[j])
        delta_j = np.append(delta_j, temp_delta_j)
        for i in range(w_1.shape[0]):
          w_1[i,j] = w_1[i,j] + eta * temp_delta_j * x[e,i]

      # print('delta_j:')
      # print(delta_j)
      # print(delta_j.shape)

      

      # for j in range(w_2.shape[0]):
      #   for k in range(w_2.shape[1]):
      #     w_2[j,k] = w_2[j,k] + eta * delta_k[k] * z[j]
      
      # for i in range(w_1.shape[0]):
      #   for j in range(w_1.shape[1]):
      #     delta_j = np.sum(np.dot(delta_k,w_2[j,:]))
      #     w_1[i,j] = w_1[i,j] + eta * delta_j * sigmoidD(z[j]) * x[e,i]
      
      print(w_1[:,4][0:5])
      
    
    training_error = np.append(training_error, np.array(error/N))
    print('End of while loop %d: '%(iteration - 1))

  return training_error, w_1, w_2


In [150]:
w_1 = ini_weights_1
w_2 = ini_weights_2

w_1.shape, w_2.shape

((785, 256), (257, 10))

In [152]:
print(ini_weights_1[:,4][0:5])
print(w_1[:, 4][0:5])
print(weights_1[:, 4][0:5])

print(ini_weights_2[:,4][0:5])
print(w_2[:, 4][0:5])
print(weights_2[:, 4][0:5])

[ 0.          0.0483203   0.04777004 -0.0335369  -0.02702244]
[ 0.          0.0483203   0.04777004 -0.0335369  -0.02702244]
[-0.00316222  0.0483203   0.04777004 -0.0335369  -0.02702244]
[ 0.         -0.13938355 -0.02286084 -0.13783363  0.03205279]
[ 0.         -0.13938355 -0.02286084 -0.13783363  0.03205279]
[-0.00550895 -0.13945529 -0.02829796 -0.13790636  0.02425556]


In [153]:
# np.ones((1,10))
x_sanity_a = np.insert(x_sanity,0,np.ones((1,6)), axis = 1)
x_sanity_a.shape

(6, 785)

In [154]:
w_1.shape, w_2.shape, x_sanity_a.shape, y_sanity.shape

((785, 256), (257, 10), (6, 785), (6, 10))

In [157]:
 training_error, weig_1, weig_2 = multiTrain(w_1,w_2,x_sanity_a,y_sanity)

Iteration number:  0
Exmaple Number:  0
Error of example 0: 1.346826
[ 0.00017252 -0.13921104 -0.02268832 -0.13783363  0.03205279]
[-6.9391095e-38  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
Exmaple Number:  1
Error of example 1: 1.617409
[ 0.0002577  -0.13913284 -0.02260316 -0.13783363  0.03213797]
[-6.9391095e-38  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
Exmaple Number:  2
Error of example 2: 1.811329
[-0.0006314  -0.14002195 -0.02349226 -0.13872273  0.03213797]
[-1.1513866e-12  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
Exmaple Number:  3
Error of example 3: 2.171383
[-0.00061904 -0.14002195 -0.0234799  -0.13872273  0.03215032]
[-1.1513866e-12  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
Exmaple Number:  4
Error of example 4: 2.231606
[-0.0006016  -0.1400045  -0.02346245 -0.13870528  0.03216777]
[-1.1513494e-12  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
Exmaple Number:  5
Error of example 5: 

In [159]:
# print(ini_weights_1[:,4][0:5])
print(weig_1[:, 4][0:5])
print(weights_1[:, 4][0:5])

# comparison = (weig_1==weights_1)
# print(comparison)

[-9.3759928e-13  4.8320301e-02  4.7770038e-02 -3.3536900e-02
 -2.7022444e-02]
[-0.00316222  0.0483203   0.04777004 -0.0335369  -0.02702244]


In [168]:
print(weig_2[:, 4][0:5])
print(weights_2[:, 4][0:5])

[-0.00056726 -0.14000145 -0.02342812 -0.13867095  0.03220211]
[-0.00550895 -0.13945529 -0.02829796 -0.13790636  0.02425556]


In [169]:
x_test_a = np.insert(x_test,0,np.ones((1,10000)), axis = 1)
y_pred_built = full_forwardPass(weig_1,weig_2,x_test_a,y_test)

accuracy, loss = calculate_Accuracy(y_pred_built, y_test_values, y_test)
print(accuracy,loss)

0.0712 2.6598299619130477


In [170]:
# x_test_a = np.insert(x_test,0,np.ones((1,10000)), axis = 1)
y_pred = full_forwardPass(weights_1,weights_2,x_test_a,y_test)

accuracy, loss = calculate_Accuracy(y_pred, y_test_values, y_test)
print(accuracy,loss)

0.1306 2.6864052654368824


In [ ]:
for idx in range(len(NN.layers)):
  print(NN.get_layer(index = idx).name)

# getting ouput for all models

# # single layer wise method
# get_layer_output = K.function([NN.layers[0].input], [NN.layers[0].output])
# layer_output = get_layer_output([x_sanity])[0]
# print(layer_output[0].shape)

input = NN.input                                           # input placeholder
outputs = [layer.output for layer in NN.layers]            # all layer outputs
functors = [K.function([input], [out]) for out in outputs]   # evaluation functions

# Testing wiht input data
# test = np.random.random(784)[np.newaxis,...]
layers = [func([x_sanity]) for func in functors]
layers_1 = layers[0][0]
layers_2 = layers[1][0]

print(layers_1.shape,layers_2.shape)


dense
dense_1
(10, 256) (10, 10)
